# Instrument Classification - Part 2 : CNN

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Flatten,Activation,BatchNormalization

In [ ]:
x_train = np.load('x_trainpggbvv.npy')
x_test  = np.load('x_testpggbvv.npy')
y_train = np.load('y_trainpggbvv.npy')
y_test  = np.load('y_testpggbvv.npy')

print("Train Size:", x_train.shape)
print("Test  Size:", x_test.shape)

In [ ]:
model = Sequential()

model.add(Conv2D(64,kernel_size=(3,3),padding='same',strides=1,input_shape = x_train[0].shape))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,kernel_size=(3,3),padding='same',strides=1))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256,kernel_size=(3,3),padding='same',strides=1))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(640,kernel_size=(3,3),padding='same',strides=1))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(Dense(6))
model.add(Activation('sigmoid'))

model.summary()

In [ ]:
#model = keras.models.load_model('ICmodel_2_mix_big_more.h5')

sgd = keras.optimizers.SGD(lr=0.0001, momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
history = model.fit(x_train,y_train,epochs=2,batch_size=32,validation_data=(x_test,y_test))

plt.plot(history.history['acc'], label='Train Accuracy')
plt.plot(history.history['val_acc'], label='Test Accuracy')
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Accuracy-Loss')
plt.legend()

model.save('ICmodel_3sec.h5')